In [1]:
import psycopg2
import pandas as pd

### Create connection to database

In [24]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=hockey_test user=postgres password=password")
except psycopg2.Error as e:
    print("Error: Could not connect to database")
    print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the database")
    print(e)

conn.set_session(autocommit=True)

#### create table for dim_teams

In [25]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS dim_teams (\
                id int, \
                Division varchar,\
                Team varchar, \
                City varchar,\
                Arena varchar,\
                Capacity int,\
                Founded int,\
                Joined int,\
                General_manager varchar,\
                Head_coach varchar,\
                Captain varchar,\
                Conference varchar,\
                ABV varchar);")
    
except psycopg2.Error as e:
    print("Error: Could not create table")
    print(e)

#### Insert values into table dim_dates

In [32]:
## load DF for dates
df_teams = pd.read_csv("/Users/mattrende/Documents/Coding/hockey-reference/YouTube Project/csv/out/dim_teams.csv")
df_teams.head(2)

,ID,Division,Team,City,Arena,Capacity,Founded,Joined,General manager,Head coach,Captain,Conference,ABV
0,3,Atlantic,Boston Bruins,"Boston, Massachusetts",TD Garden,17850,1924,1924,Don Sweeney,Jim Montgomery,Brad Marchand,Eastern,BOS
1,4,Atlantic,Buffalo Sabres,"Buffalo, New York",KeyBank Center,19070,1970,1970,Kevyn Adams,Don Granato,Kyle Okposo,Eastern,BUF


In [29]:
teams_insert = ("""INSERT INTO dim_teams(
               id,
               Division,
               Team,
               City,
               Arena,
               Capacity,
               Founded,
               Joined,
               General_manager,
               Head_coach,
               Captain,
               Conference,
               ABV)
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
               """)

In [30]:
try:
    for i, row in df_teams.iterrows():
        #print(list(row))
        cur.execute(teams_insert, list(row))

except psycopg2.Error as e:
    print("Error: could not insert rows")
    print(e)

#### Validate rows were entered

In [19]:
try: 
    cur.execute("SELECT * From dim_teams;")

except psycopg2.Error as e:
    print("Select Error")
    print(e)

row = cur.fetchone()
while row:
    #print(row)
    row = cur.fetchone()

### Close cursor and connection

In [21]:
cur.close()
conn.close()